# **Instruct Lab Quick Start Guide 2**

## **Generating synthetic dataset and Fine-Tuning the model**
### **Before you begin**
If you are going to fine-tune Granite model or any model of that size (7B) you need a **GPU instance with at least 24GB VRAM**.

Make sure your working directory is **persist-vol**. This volume is a mounted external volume to provide data persistance. 

If you are running this on a runpod cloud GPU instance, data is not going to be persisted across instance reboots as cloud instances are ephemeral. 

So make sure you use **persist-vol** directory if you need to save the data. 

Also make sure that your Jupyter Notebook Guides (.ipynb) are located in **/home/instructlab/persist-vol** as otherwise some steps in this guide **might not work**

Run the command below to see where is your working directory, it should be **/home/instructlab/persist-vol** 

In [18]:
!pwd

## **STEP 1**
### Download Granite model 

In [17]:
!ilab model download \
--repository instructlab/granite-7b-lab-GGUF \
--filename granite-7b-lab-Q4_K_M.gguf

## **STEP 2**
### Configure Instruct Lab to use Granite model 
We are going to change config.yaml to use Granite for chat and serving 

In [16]:
!sed -i '/^\s*model: models\/merlinite-7b-lab-Q4_K_M.gguf\s*$/s//  model: models\/granite-7b-lab-Q4_K_M.gguf/' config.yaml
!sed -i '/^\s*model: models\/merlinite-7b-lab-Q4_K_M.gguf\s*$/s//  model: models\/granite-7b-lab-Q4_K_M.gguf/' config.yaml
!sed -i '/^\s*host_port: 127\.0\.0\.1:8000\s*$/s//  host_port: 0.0.0.0:8000/' config.yaml
!sed -i '/^\s*model_path: models\/merlinite-7b-lab-Q4_K_M.gguf\s*$/s//  model_path: models\/granite-7b-lab-Q4_K_M.gguf/' config.yaml

Check the configuration file and confirm Granite model has been selected as default

In [15]:
!cat config.yaml

## **STEP 3**
### To test if Granite model works, start model serving and open chat 
```bash
# Terminal window 1 
ilab model serve 

# Terminal window 2
ilab model chat 

```

## **STEP 4**
### Clone git repo with a sample files 
Within taxonomy directory, which is essentialy a way to structure your training data, we have these main directories: 
  - **knowledge** - directory that will contain qna.yaml document with factualy data and question/answer pairs.
  - **compositional_skills** -
  - **foundational_skills** - 

For the simplicity of this demo we will use just knowledge directory and will populate it with existing data from my GitHub repo. 
Run the cell below. 

In [14]:
!mkdir -p /home/instructlab/persist-vol/taxonomy/knowledge/fintech
!cd /home/instructlab/persist-vol/taxonomy/knowledge/fintech \
&& git clone https://github.com/vitalijneverkevic/thoughtmachine.git

Inspect the the knowledge directory 

In [13]:
!ls -lah /home/instructlab/persist-vol/taxonomy/knowledge/fintech/thoughtmachine

Inspect qna.yaml

In [12]:
!cat /home/instructlab/persist-vol/taxonomy/knowledge/fintech/thoughtmachine/qna.yaml

## About this dataset 
This dataset is about a company called Thought Machine, which is a FinTech company specilizing in providing modern software for internet banking. 
While you have Granit model running and chat window open ask the following questions about Thought Machine: 
  - Who is the CEO of Thought Machine and what is his background?
  - Where did Paul Taylor of Thought Machine study?
  - What role Paul Taylor holds at Thought Machine?

You will see that Granit has some idea about Thought Machine, but sometimes it provides wrong information about who is the current CEO, wrong on certain specifics of the background of current CEO. 
Perhaps some of the data the model was trained on is now obsolete. 

So what we need to do is fine-tune the model and teach it so it's up to date. 

## **STEP 5**
### Launch synthetic data generation 
For synthetic data generation to work make sure you have model server running. 

In [11]:
!ilab data generate \
--model models/granite-7b-lab-Q4_K_M.gguf \
--num-cpus 4 \
--num-instructions 50 \
--taxonomy-path /home/instructlab/persist-vol/taxonomy/knowledge/fintech/thoughtmachine/qna.yaml

## **STEP 5**
### Launch model fine-tuning
Before you launch fine-tuning make sure you have model serving turned off.
Also the output is better if you run this from the terminal, you can run this from the cell below, but you will have to scroll continually to observe the output. 

In [19]:
!ilab model train \
--iters 50 \
--num-epochs 2 \
--device cuda